# Evaluate

> Evaluating and benchmarking the quality of CF explanations.

In [ ]:
# | include: false
# | default_exp evaluate


In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *
from nbdev import show_doc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# | export
from __future__ import annotations
from cfnet.import_essentials import *
from cfnet.train import train_model, TrainingConfigs
from cfnet.datasets import TabularDataModule
from cfnet.utils import accuracy, proximity
from cfnet.methods.base import BaseCFModule, BaseParametricCFModule, BasePredFnCFModule
from cfnet.methods.counternet import CounterNet
from copy import deepcopy
from sklearn.neighbors import NearestNeighbors


In [ ]:
#| export
#| hide
@dataclass
class Explanation:
    """Generated CF Explanations class."""
    cf_name: str  # cf method's name
    data_module: TabularDataModule  # data module
    cfs: jnp.DeviceArray  # generated cf explanation of `X`
    total_time: float  # total runtime
    pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray]  # predict function
    dataset_name: str = str()  # dataset name
    X: jnp.ndarray = None  # input
    y: jnp.ndarray = None  # label

    def __post_init__(self):
        if self.data_module:
            if self.dataset_name == str():
                self.dataset_name = self.data_module.data_name
            test_X, label = self.data_module.test_dataset[:]
            if self.X is None:
                self.X = test_X
            if self.y is None:
                self.y = label

CFExplanationResults = Explanation

In [ ]:
show_doc(Explanation)

<h2 id="Explanation" class="doc_header"><code>class</code> <code>Explanation</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Explanation</code>(**`cf_name`**:`str`, **`data_module`**:`TabularDataModule`, **`cfs`**:`DeviceArray`, **`total_time`**:`float`, **`pred_fn`**:`DeviceArray]`, **`dataset_name`**:`str`=*`''`*, **`X`**:`ndarray`=*`None`*, **`y`**:`ndarray`=*`None`*)

Generated CF Explanations class.

Arguments to `Explanation`:

* `cf_name`: cf method's name
* `dataset_name`: dataset name
* `X`: input
* `y`: label
* `cfs`: generated cf explanation of `X`
* `total_time`: total runtime
* `pred_fn`: predict function with only one input argument, 
and output a label (i.e., its format is `y=pred_fn(x)`).
* `data_module`: data module


## Generating CF Explanation Results


In [ ]:
#| exporti
def _validate_configs(
    cf_module: BaseCFModule,
    datamodule: TabularDataModule,
    pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray] = None,
    t_configs=None
):
    if (pred_fn is None) and (not isinstance(cf_module, BasePredFnCFModule)):
        warnings.warn(f"`{type(cf_module).__name__}` is not a subclass of `BasePredFnCFModule`."
            "This might cause problems as you set `pred_fn=None`, "
            f"which infers that `{type(cf_module).__name__}` has an attribute `pred_fn`.")


def _prepare_module(
    cf_module: BaseCFModule,
    datamodule: TabularDataModule
):
    cf_module.update_cat_info(datamodule)
    return cf_module

def _train_parametric_module(
    cf_module: BaseCFModule,
    datamodule: TabularDataModule,
    t_configs=None
):
    print(f'{type(cf_module).__name__} contains parametric models. '
        'Starts training before generating explanations...')
    cf_module.train(datamodule, t_configs)
    return cf_module

In [ ]:
#| exporti
def _check_aux_pred_fn_args(pred_fn_args: dict | None):
    if pred_fn_args is None:
        return dict()
    elif isinstance(pred_fn_args, dict):
        return pred_fn_args
    else:
        raise ValueError(f'`pred_fn_args` should be a `dict`,',
            f'but got `{type(pred_fn_args).__name__}`')

class _AuxPredFn:
    def __init__(self, pred_fn, pred_fn_args: dict | None):
        self.pred_fn = pred_fn
        self.fn_args = deepcopy(_check_aux_pred_fn_args(pred_fn_args))

    def __call__(self, x: jnp.DeviceArray) -> jnp.DeviceArray:
        return self.pred_fn(x, **self.fn_args)


def _check_pred_fn(
    pred_fn: callable | None, 
    cf_module: BaseCFModule
) -> callable:
    if pred_fn is None:
        try:
            pred_fn = cf_module.pred_fn
        except AttributeError:
            raise AttributeError(
                    "`generate_cf_explanations` is incorrectly configured."
                    f"It is supposed to be `pred_fn != None`,"
                    f"or `{type(cf_module).__name__}` has attribute `pred_fn`."
                    f"However, we got `pred_fn={pred_fn}`, "
                    f"and `{type(cf_module).__name__}` has not attribute `pred_fn`."
            )
    elif isinstance(cf_module, BasePredFnCFModule):
        # override pred_fn if `cf_module` has `pred_fn`
        pred_fn = cf_module.pred_fn
    return pred_fn

In [ ]:
#| export
def generate_cf_explanations(
    cf_module: BaseCFModule, # CF Explanation Module
    datamodule: TabularDataModule, # Data Module
    pred_fn: callable = None, # Predictive function
    t_configs: TrainingConfigs = None, # training configs for `BaseParametricCFModule`
    pred_fn_args: dict = None # auxiliary arguments for `pred_fn` 
) -> Explanation:
    """Generate CF explanations."""

    _validate_configs(cf_module, datamodule, pred_fn, t_configs)
    cf_module = _prepare_module(cf_module, datamodule)

    if isinstance(cf_module, BaseParametricCFModule):
        cf_module = _train_parametric_module(
            cf_module, datamodule, t_configs=t_configs
        )
    X, _ = datamodule.test_dataset[:]
    
    # create `pred_fn` which only takes `x` as an input
    if pred_fn is not None:
        pred_fn = _AuxPredFn(pred_fn, pred_fn_args=pred_fn_args)

    # generate cfs
    current_time = time.time()
    cfs = cf_module.generate_cfs(X, pred_fn=pred_fn)
    total_time = time.time() - current_time
    # check pred_fn
    pred_fn = _check_pred_fn(pred_fn, cf_module)

    return Explanation(
        cf_name=cf_module.name,
        data_module=datamodule,
        cfs=cfs,
        total_time=total_time,
        pred_fn=pred_fn,
    )


The `pred_fn` in `generate_cf_explanations` is a model's prediction function. 
The general format is `y = pred_fn(x, **pred_fn_args)`. 
If `pred_fn` is not parameterized by other variables (except input `x`), 
then `pred_fn_args` is set to `None`, which is the default setting.
Otherwise, you should pass these argument as a `dict`.

For example, we have a simple linear function

In [ ]:
def linear_pred_fn(x: jnp.DeviceArray, params: jnp.DeviceArray):
    return x @ params

To pass `linear_pred_fn` to `generate_cf_explanations`, 
we can either create an auxiliary function of `linear_pred_fn`,
or pass `params` into `pred_fn_args`.

Assuming we now have the input `x` and `params`:

In [ ]:
x = jax.random.normal(random.PRNGKey(0), shape=(5, 10)) # input
params = jnp.ones((10, 1)) # params

1. Create an auxillary function (Not recommended)

```python
aux_linear_pred_fn = lambda x: linear_pred_fn(x, params)
explanations = generate_cf_explanations(
    cf_module, datamodule, aux_linear_pred_fn
)
```

This approach could work, but if `params` is changed, 
`explanations.pred_fn` might not work as expected.

2. Pass `params` into `pred_fn_args`

```python
explanations = generate_cf_explanations(
    cf_module, datamodule, linear_pred_fn, 
    pred_fn_args=dict(params=params)
)
```

This is a recommended approach as we will deepcopy `params` inside `generate_cf_explanations`.


The `pred_fn` in `explanations` only takes `x: jnp.DeviceArray` as an input.
For example, to make predictions, we use

```python
y = explanations.pred_fn(x)
```

In [ ]:
#| export
#| hide
@deprecated(removed_in='0.1.0', deprecated_in='0.0.9')
def generate_cf_results_local_exp(
    cf_module: BaseCFModule,
    dm: TabularDataModule,
    pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray],
) -> CFExplanationResults:
    return generate_cf_explanations(cf_module, dm, pred_fn=pred_fn)


@deprecated(removed_in='0.1.0', deprecated_in='0.0.9')
def generate_cf_results_cfnet(
    cf_module: CounterNet,
    dm: TabularDataModule,
    params: hk.Params = None,  # params of `cf_module`
    rng_key: random.PRNGKey = None,
) -> CFExplanationResults:
    return generate_cf_explanations(cf_module, dm, pred_fn=None)


## Evaluating Metrics

In [ ]:
# | export
def compute_predictive_acc(cf_results: CFExplanationResults):
    X, y = cf_results.data_module.test_dataset[:]
    pred_fn = cf_results.pred_fn

    y_pred = pred_fn(X).reshape(-1, 1)
    label = y.reshape(-1, 1)
    return accuracy(jnp.round(y_pred), label).item()


def compute_validity(cf_results: CFExplanationResults):
    X, y = cf_results.data_module.test_dataset[:]
    pred_fn = cf_results.pred_fn

    y_pred = pred_fn(X).reshape(-1, 1).round()
    y_prime = 1 - y_pred
    cf_y = pred_fn(cf_results.cfs).reshape(-1, 1).round()
    return accuracy(y_prime, cf_y).item()


def compute_proximity(cf_results: CFExplanationResults):
    X, y = cf_results.data_module.test_dataset[:]
    return proximity(X, cf_results.cfs).item()


def compute_sparsity(cf_results: CFExplanationResults):
    X, y = cf_results.data_module.test_dataset[:]
    cfs = cf_results.cfs
    cat_idx = cf_results.data_module.cat_idx
    # calculate sparsity
    cat_sparsity = proximity(X[:, cat_idx:], cfs[:, cat_idx:]) / 2
    cont_sparsity = jnp.linalg.norm(
        jnp.abs(X[:, :cat_idx] - cfs[:, :cat_idx]), ord=0, axis=1
    ).mean()
    return cont_sparsity + cat_sparsity


def compute_manifold_dist(cf_results: CFExplanationResults):
    X, y = cf_results.data_module.test_dataset[:]
    cfs = cf_results.cfs
    knn = NearestNeighbors()
    knn.fit(X)
    nearest_dist, nearest_points = knn.kneighbors(cfs, 1, return_distance=True)
    return jnp.mean(nearest_dist).item()


def get_runtime(cf_results: CFExplanationResults):
    return cf_results.total_time


def _create_second_order_cfs(cf_results: CFExplanationResults, threshold: float = 2.0):
    X, y = cf_results.data_module.test_dataset[:]
    cfs = cf_results.cfs
    scaler = cf_results.data_module.normalizer
    cat_idx = cf_results.data_module.cat_idx

    # get normalized threshold = threshold / (max - min)
    data_range = scaler.data_range_
    thredshold_normed = threshold / data_range

    # select continous features
    x_cont = X[:, :cat_idx]
    cf_cont = cfs[:, :cat_idx]
    # calculate the diff between x and c
    cont_diff = jnp.abs(x_cont - cf_cont) <= thredshold_normed
    # new cfs
    cfs_cont_hat = jnp.where(cont_diff, x_cont, cf_cont)

    cfs_hat = jnp.concatenate((cfs_cont_hat, cfs[:, cat_idx:]), axis=-1)
    return cfs_hat


def compute_so_validity(cf_results: CFExplanationResults, threshold: float = 2.0):
    cfs_hat = _create_second_order_cfs(cf_results, threshold)
    cf_results_so = deepcopy(cf_results)
    cf_results_so.cfs = cfs_hat
    return compute_validity(cf_results_so)


def compute_so_proximity(cf_results: CFExplanationResults, threshold: float = 2.0):
    cfs_hat = _create_second_order_cfs(cf_results, threshold)
    cf_results_so = deepcopy(cf_results)
    cf_results_so.cfs = cfs_hat
    return compute_proximity(cf_results_so)


def compute_so_sparsity(cf_results: CFExplanationResults, threshold: float = 2.0):
    cfs_hat = _create_second_order_cfs(cf_results, threshold)
    cf_results_so = deepcopy(cf_results)
    cf_results_so.cfs = cfs_hat
    return compute_sparsity(cf_results_so)


In [ ]:
# | export
metrics2fn = {
    "acc": compute_predictive_acc,
    "validity": compute_validity,
    "proximity": compute_proximity,
    "runtime": get_runtime,
    "manifold_dist": compute_manifold_dist,
    "so_validity": compute_so_validity,
    "so_proximity": compute_so_proximity,
    "so_sparsity": compute_so_sparsity,
}


In [ ]:
# | export
DEFAULT_METRICS = ["acc", "validity", "proximity"]


def evaluate_cfs(
    cf_results: CFExplanationResults,
    metrics: Optional[Iterable[str]] = None,
    return_dict: bool = True,
    return_df: bool = False,
):
    cf_name = cf_results.cf_name
    data_name = cf_results.data_module.data_name
    result_dict = {(data_name, cf_name): dict()}
    if metrics is None:
        metrics = DEFAULT_METRICS

    for metric in metrics:
        result_dict[(data_name, cf_name)][metric] = metrics2fn[metric](cf_results)
    result_df = pd.DataFrame.from_dict(result_dict, orient="index")
    if return_dict and return_df:
        return (result_dict, result_df)
    elif return_dict or return_df:
        return result_df if return_df else result_dict


In [ ]:
# | export
def benchmark_cfs(
    cf_results_list: Iterable[CFExplanationResults],
    metrics: Optional[Iterable[str]] = None,
):
    dfs = [
        evaluate_cfs(
            cf_results=cf_results, metrics=metrics, return_dict=False, return_df=True
        )
        for cf_results in cf_results_list
    ]
    return pd.concat(dfs)

## How to evaluate a CF Explanation Module

In [ ]:
from cfnet.module import PredictiveTrainingModule
from cfnet.utils import load_json

In [ ]:
configs = load_json('assets/configs/data_configs/adult.json')
m_configs = configs['mlp_configs']
data_configs = configs['data_configs']
data_configs['sample_frac'] = 0.1

t_configs = {
    'n_epochs': 10,
    'monitor_metrics': 'val/val_loss',
    'seed': 42,
    "batch_size": 256
} 

We first train a model

In [ ]:
training_module = PredictiveTrainingModule(m_configs)
dm = TabularDataModule(data_configs)

params, opt_state = train_model(
    training_module, 
    dm, 
    t_configs
)
pred_fn = lambda x, params, prng_key: \
    training_module.forward(params, prng_key, x, is_training=False)

Epoch 9: 100%|██████████| 10/10 [00:00<00:00, 86.11batch/s, train/train_loss_1=0.0701]


Now, we can start to benchmark different methods

In [ ]:
from cfnet.methods import VanillaCF, CounterNet

Generate CF explanations for `VanillaCF`

In [ ]:
#| slow
vanillacf = VanillaCF(dict(n_steps=1000, lr=0.001))
vanillacf_exp = generate_cf_explanations(
    vanillacf, dm, pred_fn,
    pred_fn_args=dict(params=params, prng_key=random.PRNGKey(0))
)

100%|██████████| 1000/1000 [00:05<00:00, 195.16it/s]


In [ ]:
#| slow
assert vanillacf_exp.cf_name == vanillacf.name
assert vanillacf_exp.dataset_name == dm.data_name
assert vanillacf_exp.X.shape == vanillacf_exp.cfs.shape
assert vanillacf_exp.pred_fn(vanillacf_exp.X).shape == vanillacf_exp.y.shape

Generate CF explanations for `CounterNet`

In [ ]:
#| slow
counternet = CounterNet()
counternet_exp = generate_cf_explanations(counternet, dm, pred_fn=None)

CounterNet contains parametric models. Starts training before generating explanations...


Epoch 99: 100%|██████████| 20/20 [00:00<00:00, 162.73batch/s, train/train_loss_1=0.0673, train/train_loss_2=0.000218, train/train_loss_3=0.0868] 


Note that `CounterNet` contains a predictive module, so we set `pred_fn=None`

In [ ]:
#| slow
assert counternet_exp.cf_name == counternet.name
assert counternet_exp.dataset_name == dm.data_name
assert counternet_exp.X.shape == counternet_exp.cfs.shape
assert counternet_exp.pred_fn(counternet_exp.X).shape == counternet_exp.y.shape
assert counternet_exp.pred_fn == counternet.pred_fn

If `cf_module` is a subclass of `BasePredFnCFModule` (e.g., `CounterNet`),
the `pred_fn` in `Explanation` will be set to `cf_module.pred_fn`,
and the `pred_fn` argument passed `generate_cf_explanations` will be ignored.

In [ ]:
#| slow
counternet_exp_1 = generate_cf_explanations(counternet, dm, pred_fn=pred_fn)
assert counternet_exp_1.pred_fn != pred_fn
assert counternet_exp_1.pred_fn == counternet.pred_fn

CounterNet contains parametric models. Starts training before generating explanations...


Epoch 99: 100%|██████████| 20/20 [00:00<00:00, 175.02batch/s, train/train_loss_1=0.068, train/train_loss_2=4.58e-5, train/train_loss_3=0.0856]   


Now, we can compute metrics for benchmarking different CF explanation methods.

In [ ]:
#| slow
evaluate_cfs(vanillacf_exp, return_df=True)[1]

,,acc,validity,proximity
adult,VanillaCF,0.817221,0.279081,10.302892


In [ ]:
#| slow
evaluate_cfs(counternet_exp, return_df=True)[1]

,,acc,validity,proximity
adult,CounterNet,0.82017,0.96622,5.716282
